In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


In [2]:
from acnportal.acnsim.events import Event, EventQueue, PluginEvent, UnplugEvent, RecomputeEvent
from acnportal.acnsim.interface import Interface
from acnportal.acnsim.models import EV, Battery, EVSE
from acnportal.acnsim.network.sites import caltech_acn, jpl_acn
from acnportal.acnsim.simulator import Simulator
import gym
import numpy as np

from sustaingym.envs.evcharging.actions import get_action_space, to_schedule
from sustaingym.envs.evcharging.ev_charging import EVChargingEnv
from sustaingym.envs.evcharging.event_generation import AbstractTraceGenerator
from sustaingym.envs.evcharging.observations import get_observation_space, get_observation
from sustaingym.envs.evcharging.rewards import get_rewards
from sustaingym.envs.evcharging.utils import MINS_IN_DAY, DATE_FORMAT


In [3]:
acn = caltech_acn()
acn.station_ids[:7]

['CA-308', 'CA-508', 'CA-303', 'CA-513', 'CA-310', 'CA-506', 'CA-316']

In [20]:
class TestGenerator(AbstractTraceGenerator):
    def __init__(self,
                 site: str = "",
                 period: int = 5,
                 recompute_freq: int = 2,
                 date_range = None,
                 requested_energy_cap: float = 100):

        super().__init__(site, period, recompute_freq, date_range, requested_energy_cap)
    
    def update_day(self):
        return
    
    def get_event_queue(self):
        events = []
        for a, d, s in zip([10], [30], ['CA-508',]):
            req_energy = 30.0
            battery = Battery(capacity=100, init_charge=max(0, 100-req_energy), max_power=100)
            ev = EV(arrival=a, departure=d, requested_energy=req_energy, station_id=s, session_id=str(a), battery=battery, estimated_departure=a+50)

            events.append(PluginEvent(a, ev))
        events.append(RecomputeEvent(0))
        eq = EventQueue(events)
        return eq, 3

In [21]:
tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
eq, _ = tg.get_event_queue()

In [22]:
eq.get_current_events(45)

[acnportal.acnsim.events.event.RecomputeEvent(timestamp=0, event_type=<str object at 0x7f8d0192a170>, precedence=20),
 acnportal.acnsim.events.event.PluginEvent(timestamp=10, event_type=<str object at 0x7f8d0192a070>, precedence=10, ev=<acnportal.acnsim.models.ev.EV object at 0x7f8d0d3b3790>)]

In [23]:
acn = caltech_acn()
acn.magnitudes

array([ 80.        ,  80.        , 416.66666667, 416.66666667,
       416.66666667, 180.50541516, 180.50541516, 180.50541516])

In [24]:
np.maximum(np.abs(acn.constraint_current(np.ones(shape=(54,), ) * 16, linear=True)) - acn.magnitudes, 0)

array([48., 48.,  0.,  0.,  0.,  0.,  0.,  0.])

In [25]:
(acn.constraint_matrix @ np.ones(shape=(54,), ) * 16)

array([ 128.,  128.,  192., -192.,    0.,   48.,  -96.,   48.])

In [26]:
from collections import defaultdict

# send 16 A in

tg = TestGenerator(site='caltech', date_range=['2018-11-07', '2018-11-13'])
env = EVChargingEnv(tg, action_type='discrete', verbose=2)
for _ in range(1):
    observation = env.reset()

    rewards = 0.
    done = False
    i = 0
    action = np.ones(shape=(54,), ) * 2
    d = defaultdict(list)
    while not done:
        observation, reward, done, info = env.step(action)
        rewards += reward
        i += 1

    print()
    print()
    print("total iterations: ", i)
    print("environment reward: ", rewards)


Simulating 3 plug in events. 
REWARD FUNCTION:  0 0 10 50 -4800.0
REWARD FUNCTION:  0 10 30 100 -9600.0
REWARD FUNCTION:  10 30 288 1290 -123840.0


total iterations:  3
environment reward:  -138240.0


/home/ubuntu/.local/lib/python3.9/site-packages/acnportal/acnsim/simulator.py:277: UserWarning: Invalid schedule provided at iteration 0. Max violation is 145.84198993323565 A on Secondary A at time index 0.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/acnportal/acnsim/simulator.py:277: UserWarning: Invalid schedule provided at iteration 1. Max violation is 145.84198993323565 A on Secondary A at time index 0.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/acnportal/acnsim/simulator.py:277: UserWarning: Invalid schedule provided at iteration 2. Max violation is 145.84198993323565 A on Secondary A at time index 0.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/acnportal/acnsim/simulator.py:277: UserWarning: Invalid schedule provided at iteration 3. Max violation is 145.84198993323565 A on Secondary A at time index 0.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/acnportal/acnsim/simulator.py:277: UserWarning: Inva

In [81]:
#-7853.846153846154
#-7053.846153846153

[]

In [185]:
# 0  50 11920.0  12000
# 0 100 13769.23 16000
# 0 150 13769.23 20000
# 0 200 13769.23 24000

11200

11200